In [1]:
from pyspark.sql import SparkSession
import getpass
username = getpass.getuser()
spark = SparkSession. \
builder. \
config('spark.ui.port', '0'). \
config('spark.shuffle.useOldFetchProtocol', 'true'). \
config("spark.sql.warehouse.dir", f"/user/{username}/warehouse"). \
enableHiveSupport(). \
master('yarn'). \
getOrCreate()

In [2]:
orders_schema = "order_id long , order_date date, customer_id long,order_status string"

In [3]:
orders_df = spark.read \
.format("csv") \
.schema(orders_schema) \
.load("/public/trendytech/orders/orders_1gb.csv")

In [20]:
spark.catalog.currentDatabase()

'default'

In [21]:
spark.sql("use itv006277_cachingdemo_db")

""


In [ ]:
spark.catalog.currentDatabase()

In [22]:
spark.catalog.isCached("itv006277_cachingdemo_db.itv006277_orders1")

True

In [23]:
spark.catalog.clearCache()

In [30]:
spark.sql("create database itv006277_caching_demo_ext")

""


### Note: You can use any orders file as path for csv location for creating a table 

Creating External Table

In [53]:
spark.sql("create table itv006277_caching_demo_ext.itv006277_orders_ext(order_id long, order_date string, customer_id long, order_status string) using csv location '/user/itv006277/orders/'")

""


In [54]:
spark.sql("select * from itv006277_caching_demo_ext.itv006277_orders_ext").show()

+--------+--------------------+-----------+---------------+
|order_id|          order_date|customer_id|   order_status|
+--------+--------------------+-----------+---------------+
|       1|2013-07-25 00:00:...|      11599|         CLOSED|
|       2|2013-07-25 00:00:...|        256|PENDING_PAYMENT|
|       3|2013-07-25 00:00:...|      12111|       COMPLETE|
|       4|2013-07-25 00:00:...|       8827|         CLOSED|
|       5|2013-07-25 00:00:...|      11318|       COMPLETE|
|       6|2013-07-25 00:00:...|       7130|       COMPLETE|
|       7|2013-07-25 00:00:...|       4530|       COMPLETE|
|       8|2013-07-25 00:00:...|       2911|     PROCESSING|
|       9|2013-07-25 00:00:...|       5657|PENDING_PAYMENT|
|      10|2013-07-25 00:00:...|       5648|PENDING_PAYMENT|
|      11|2013-07-25 00:00:...|        918| PAYMENT_REVIEW|
|      12|2013-07-25 00:00:...|       1837|         CLOSED|
|      13|2013-07-25 00:00:...|       9149|PENDING_PAYMENT|
|      14|2013-07-25 00:00:...|       98

In [55]:
spark.sql("select count(*) from itv006277_caching_demo_ext.itv006277_orders_ext").show()

+--------+
|count(1)|
+--------+
|    1000|
+--------+



How Do i check the external table ?

Under the  type columns external 

In [56]:
spark.sql("describe extended itv006277_caching_demo_ext.itv006277_orders_ext").show()

+--------------------+--------------------+-------+
|            col_name|           data_type|comment|
+--------------------+--------------------+-------+
|            order_id|              bigint|   null|
|          order_date|              string|   null|
|         customer_id|              bigint|   null|
|        order_status|              string|   null|
|                    |                    |       |
|# Detailed Table ...|                    |       |
|            Database|itv006277_caching...|       |
|               Table|itv006277_orders_ext|       |
|               Owner|           itv006277|       |
|        Created Time|Wed Nov 01 02:34:...|       |
|         Last Access|Wed Dec 31 19:00:...|       |
|          Created By|         Spark 2.4.7|       |
|                Type|            EXTERNAL|       |
|            Provider|                 csv|       |
|    Table Properties|[transient_lastDd...|       |
|            Location|hdfs://m01.itvers...|       |
|       Serd

Creating cache

In [57]:
spark.sql("cache table itv006277_caching_demo_ext.itv006277_orders_ext")

""


In [58]:
spark.sql("insert into itv006277_caching_demo_ext.itv006277_orders_ext values(111111, '2023-05-29', 222222, 'BOOKED')")

""


Cache will not hit due to we did some changes in table

In [59]:
spark.sql("select count(*) from itv006277_caching_demo_ext.itv006277_orders_ext").show()

+--------+
|count(1)|
+--------+
|    1001|
+--------+



How to refresh table ?

In [66]:
spark.sql("refresh table itv006277_caching_demo_ext.itv006277_orders_ext")

""


In [67]:
spark.catalog.refreshTable("itv006277_caching_demo_ext.itv006277_orders_ext")

In [68]:
spark.sql("select count(*) from itv006277_caching_demo_ext.itv006277_orders_ext").show()

+--------+
|count(1)|
+--------+
|    1201|
+--------+



When you insert suing insert command then spark will know that the cache is invalidated and in next subsequent use it will refresh it

but when we add or remove files in backend then spark cannot track it and we have to refresh the table manually